In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout

In [13]:
CIFAR10 = keras.datasets.cifar10

(x_train , y_train) , (x_test , y_test ) = CIFAR10.load_data()
x_train , x_test = x_train/255.0 , x_test/255.0

In [14]:
x_train[2].shape

(32, 32, 3)

# Adding batch size

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train , y_train)).batch(32).shuffle(buffer_size=1000)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test , y_test)).batch(32)

In [16]:
x_train.shape

(50000, 32, 32, 3)

In [17]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv2D_1 = Conv2D(16 , (3,3) , activation="relu" )
    self.conv2D_11 = Conv2D(32 , (3,3) , activation="relu" )
    self.conv2D_2 = Conv2D(64 , (3,3) , activation="relu" )
    self.conv2D_22 = Conv2D(128 , (3,3) , activation="relu" )
    self.maxpool2D = MaxPooling2D()
    self.dropout = Dropout(0.3)
    self.dropout2 = Dropout(0.5)
    self.flatten = Flatten()
    self.dense1 = Dense(1000 , activation="relu")
    self.dense2 = Dense(10 , activation="softmax")

  def call(self , x):
    x = self.conv2D_1(x)
    x = self.conv2D_11(x)
    x = self.maxpool2D(x)
    x = self.dropout(x)
    x = self.conv2D_2(x)
    x = self.conv2D_22(x)
    x = self.maxpool2D(x)
    x = self.dropout(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    return x

model = MyModel()

In [18]:
# Main loss function
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [19]:
# Metrics

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# Train

In [20]:
@tf.function
def train(images , labels):
    with tf.GradientTape() as tape :
      prediction = model(images)
      loss = loss_function(y_true=labels , y_pred=prediction)

    gradients = tape.gradient(loss , model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars= zip(gradients , model.trainable_variables))

    train_loss(loss)
    train_acc(labels, prediction)

In [21]:
@tf.function
def test(images , labels):
  prediction = model(images)
  loss = loss_function(y_true=labels , y_pred=prediction)
  test_loss(loss)
  test_acc(labels , prediction)

In [22]:
epochs = 25
for epoch in range(epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  #train
  for images , labels in train_dataset :
    train(images , labels)

  # test
  for images , labels in test_dataset :
    test(images , labels)


  print("epoch:" , epoch + 1  ,
        f"Train Loss: {train_loss.result()}" ,
        f"Train Acc: {train_acc.result()}" ,
        f"Test loss: {test_loss.result()}",
        f"Test Acc: {test_acc.result()}")

epoch: 1 Train Loss: 1.7218892574310303 Train Acc: 0.37770000100135803 Test loss: 1.4796860218048096 Test Acc: 0.46389999985694885
epoch: 2 Train Loss: 1.3854135274887085 Train Acc: 0.5043399930000305 Test loss: 1.2721941471099854 Test Acc: 0.5482000112533569
epoch: 3 Train Loss: 1.2347691059112549 Train Acc: 0.5632200241088867 Test loss: 1.1947051286697388 Test Acc: 0.5748000144958496
epoch: 4 Train Loss: 1.1271772384643555 Train Acc: 0.6043000221252441 Test loss: 1.1196439266204834 Test Acc: 0.6057999730110168
epoch: 5 Train Loss: 1.0395210981369019 Train Acc: 0.6358799934387207 Test loss: 1.047363042831421 Test Acc: 0.6326000094413757
epoch: 6 Train Loss: 0.9664450287818909 Train Acc: 0.6621800065040588 Test loss: 0.9778339862823486 Test Acc: 0.6553000211715698
epoch: 7 Train Loss: 0.8990380764007568 Train Acc: 0.6891599893569946 Test loss: 1.0230090618133545 Test Acc: 0.6435999870300293
epoch: 8 Train Loss: 0.83841472864151 Train Acc: 0.7106199860572815 Test loss: 0.939471542835235